In [1]:
import numpy as np
from scipy import stats
from sklearn.gaussian_process import GaussianProcessClassifier, kernels
from matplotlib import pyplot as pl
from matplotlib import cm
import csv


def merge_dicts(*dict_args):
    result = {}  # empty dict
    for dictionary in dict_args:
        # go through dict, update master dict
        result.update(dictionary)
    return result  # return master dict

## Setup
This code will take the data on amplitude of the two frequencies (already
generated by matlab) and classify the perceptual response during rivalry
based on training data from the **test** set

You can define who to include include in this analysis run in the next cell.
You can also set some of the other parameters that might change from setup
to setup:

In [2]:
participants = ['stratus' + str(number) for number in [101, 102, 103, 104, 105,
                                                       106, 109, 112, 114, 116,
                                                       117, 118, 119, 121, 122,
                                                       123, 124, 125, 126, 127,
                                                       128, 129, 130, 131, 132]]

# There are three possible perceptual reports
perceptindices = {'lowfreq': -1, 'mixed': 0, 'highfreq': 1}
# Use these colors for plots:
colors = {'lowfreq': 'blue', 'mixed': 'gray', 'highfreq': 'red'}

## Data Cleaning
First we define a function that downsamples the data. This is done "safely"
so we never average data over points where a different percept was reported.
This is due to the memory limitations that the machine learning algorithm has.

In [3]:
def cleandata(data, downsamplefactor=20):
    assert data.shape[1] == 3, "Data not in the right format."

    # trim the data of all timepoints where nothing was reported
    # (this usually also removes the beginning and end of trials)
    data = data[np.logical_not(np.isnan(data[:, 2])), :]

    # downsample the data (to avoid memory issues!)
    # average over timebins (n=downsamplefactor) with same percept
    data = np.array([
        np.mean(
            data[(index - downsamplefactor / 2):
                 (index + downsamplefactor / 2), :][
                # logical index
                np.equal(data[
                    (index - downsamplefactor / 2):
                    (index + downsamplefactor / 2), 2
                ], data[index, 2]), :
            ],
            axis=0)
        for index in np.arange(downsamplefactor / 2, data.shape[0],
                               downsamplefactor)])
    # split into x and y
    x = data[:, 0:2]
    y = data[:, 2]
    return x, y

## Model Training
The next section is where the model is actually trained. Set changes to
the kernel below. You can also change whether the script produces a plot
for each subject and what those plots look like.

In [4]:
def trainPredict(subjectid, makeplot=False):
    print("testing participant " + subjectid)
    # Load training data from the file matlab generates
    traindata = np.genfromtxt('csvdata/' + subjectid +
                              '_sim.csv', delimiter=',',
                              missing_values=['NaN', 'nan'],
                              filling_values=None)
    # Clean + downsample this data
    trainx, trainy = cleandata(traindata, downsamplefactor=20)

    # Train a Gaussian Process
    anisokern = kernels.RBF()  # default kernel
    gp = GaussianProcessClassifier(kernel=anisokern)  # Initialize the GPC
    gp.fit(trainx, trainy)  # train this class on the data
    trainx = trainy = None  # Discard all training data to preserve memory

    # load test data
    testdata = np.genfromtxt('csvdata/' + subjectid +
                             '_rival.csv', delimiter=',',
                             missing_values=['NaN', 'nan'],
                             filling_values=None)

    testx, testy = cleandata(testdata, downsamplefactor=4)  # clean data

    testdata = None  # clear from memory
    # work out percentage in percept for each data point:
    percentages = assign_percentage(testy)

    # get a prediction for all points in the test data:
    predicty = gp.predict(testx)
    proby = gp.predict_proba(testx)

    if makeplot:
        summaryplot(participant, testx, testy, predicty, proby, gp)

    # Summarise prediction by reported percept
    meanprediction = {'mean' + percept:
                      proby[testy == value, 1].mean()
                      for percept, value in perceptindices.iteritems()}
    predictiondev = {'stdev' + percept:
                     proby[testy == value, 1].std()
                     for percept, value in perceptindices.iteritems()}
    predictionaccuracy = {'acc' + percept:
                          (predicty[testy == value] ==
                           testy[testy == value]).mean()
                          for percept, value in perceptindices.iteritems()}
    # Summarise prediction by percentage in percept
    predictioncourse = {percept:
                        [proby[(testy == value) &
                               (percentages < cutoff) &
                               (percentages > cutoff - 0.1), 1].mean()
                         for cutoff in np.linspace(0.1, 1, 10)]
                        for percept, value in perceptinidices.iteritems()}

    # Only return the summarised data
    return meanprediction, predictiondev, predictionaccuracy, predictioncourse

In order to aid visualisation, we need to create a function that creates a
plot summarising the results of our gaussian classifier for each participant:

In [5]:
def summaryplot(participant, testx, testy, predicty, proby, gp):
    # make plot here
    fig = pl.figure(1)
    # make a kernel density plot
    ax = pl.subplot2grid((2, 3), (0, 0), colspan=2, rowspan=2)
    pl.xlim(0, 1)
    xspan = np.linspace(0, 1, 200)
    for percept, value in perceptindices.iteritems():
        density = stats.gaussian_kde(proby[np.equal(testy, value), 1])
        yspan = density(xspan)
        yspan[0] = 0
        yspan[-1] = 0
        pl.plot(xspan, yspan, label=percept, color=colors[percept])
        pl.fill(xspan, yspan, color=colors[percept], alpha=0.2)
    pl.legend()

    # make a plot of the kernel
    lim = 3
    res = 50
    ax = pl.subplot2grid((2, 3), (0, 2))
    # X axes:
    x1, x2 = np.meshgrid(np.linspace(- lim, lim, res),
                         np.linspace(- lim, lim, res))
    xx = np.vstack([x1.reshape(x1.size), x2.reshape(x2.size)]).T
    # Y:
    y_pred = gp.predict_proba(xx)
    y_pred = y_pred[:, 0].reshape((res, res))
    ax.axes.set_aspect('equal')
    pl.xticks([])
    pl.yticks([])
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    pl.xlabel('Frequency 1')
    pl.ylabel('Frequency 2')
    cax = pl.imshow(np.flipud(y_pred), cmap=cm.gray_r, alpha=0.8,
                    extent=(- lim, lim, - lim, lim))
    for color, value in (('b', 0.2), ('k', 0.5), ('r', 0.8)):
        cs = pl.contour(x1, x2, y_pred, [value], colors=color,
                        linestyles='dashed')
        pl.clabel(cs, fontsize=11)

    # Visualise the categorical predictions
    ax = pl.subplot2grid((2, 3), (1, 2))
    domindices = (np.equal(testy, index) for index in (-1, 1))
    correctpercentage = [np.mean(np.equal(testy[domindex],
                                          predicty[domindex]))
                         for domindex in domindices]
    pl.bar([-1.5, 0.5], correctpercentage)
    pl.ylim(0, 1)
    pl.xlim(-2, 2)
    pl.ylabel('Correctly predicted')
    pl.xticks([-1, 1])
    ax.set_xticklabels(['Low Freq', 'High Freq'])

    fig.subplots_adjust(wspace=0.5)
    fig.suptitle(participant)
    pl.savefig('graphs/' + participant + '.png')

## Behavioral report parsing
The following function is for calculating the proportions of each
percept in a given rivalry trial for each subject.

In [6]:
def compute_proportions(subjectid):
    testdata = np.genfromtxt('csvdata/' + subjectid +
                             '_rival.csv', delimiter=',',
                             missing_values=['NaN', 'nan'],
                             filling_values=None)
    testx, testy = cleandata(testdata)  # clean data
    perceptprop = {'proportion' + percept: float(np.sum(
        np.equal(testy, value)
    )) / float(testy.size)
        for percept, value in perceptindices.iteritems()}
    return perceptprop

## Tracking predictions over the course of a rivalry period
This function is for calculating the percentage of a particular rivalry
period that a given data point is in. You can then use this to bin the
probabilities by, and so trace the evolution of estimated probabilitie during
the course of a perceptual period.

In [7]:
def assign_percentage(y):
    percentages = []
    startpoint = 0
    previous = 99
    for timepoint in range(y.size):
        percept = y[timepoint]
        if percept != previous:
            previous = percept
            startpoint = timepoint
            try:
                endpoint = ((y != percept) &
                            (np.arange(y.size) >= timepoint)).nonzero()[0][0]
            except:
                endpoint = y.size
        percentages.append(float(timepoint - startpoint) /
                           float(endpoint - startpoint))
    return percentages

## Data Saving
The following cell opens a "resultfile" spreadsheet and creates a class
for writing to that spreadsheet.

trainPredict('stratus125', makeplot=False)
raise KeyboardInterrupt

In [ ]:
resultfile = open('all-results.csv', 'w+')  # open file
# open a "writer" which takes a dictionary as an argument:
resultwriter = csv.DictWriter(resultfile, delimiter=',', lineterminator='\n',
                              fieldnames=['subject'] +
                              ['mean' + percept
                               for percept in perceptindices.keys()] +
                              ['stdev' + percept
                               for percept in perceptindices.keys()] +
                              ['acc' + percept
                               for percept in perceptindices.keys()] +
                              ['proportion' + percept
                               for percept in perceptindices.keys()])
resultwriter.writeheader()  # write the column headers to the spreadsheet

This is the loop where the models get trained. Each subject's model is only
in memory once, so if you want to create more plots / write more results to
spreadsheet, make sure to change them before running this cell!

In [ ]:
try:
    for participant in participants:
        # Train the model, predict percepts:
        meanprediction, predictiondev, predictionaccuracy = trainPredict(
            participant,
            makeplot=True  # change to false if no "summary" plot is needed
        )

        # Work out the percept proportions during rivalry
        proportions = compute_proportions(participant)

        # Write to file:
        meanprediction['subject'] = participant
        all_results = merge_dicts(meanprediction, predictiondev,
                                  predictionaccuracy, proportions)
        resultwriter.writerow(all_results)
finally:
    resultfile.close()  # close file